In [1]:
import yfinance as yf
import requests
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, col, year, month, sum as spark_sum, avg as spark_avg

In [2]:
# Function Libs
def download_yfinance_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date).reset_index()
    data.columns = data.columns.droplevel(-1)
    data['Stock'] = ticker
    data['Source'] = 'yfinance'
    return data

def download_alpha_vantage_data(ticker, api_key):
    #url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={ticker}&apikey={api_key}&outputsize=full"
    url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=IBM&outputsize=full&apikey=demo"
    response = requests.get(url)
    
    data = pd.DataFrame.from_dict(response.json()['Time Series (Daily)'], orient= 'index').sort_index(axis=1)
    data.rename(columns={
        'timestamp': 'Date',
        '1. open': 'Open',
        '2. high': 'High',
        '3. low': 'Low',
        '4. close': 'Close',
        '5. adjusted close': 'Adj Close',
        '6. volume': 'Volume'
    }, inplace=True)
    data['Stock'] = ticker
    data['Source'] = 'alpha vantage'
    data['str_date'] = data.index
    data = data[['str_date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Stock', 'Source']].reset_index(drop=True)
    
    return data

In [3]:
# Configuration
from secret import credential
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

start_date = "2023-01-01"
end_date = "2024-01-01"
tickers = ['IBM']
alpha_vantage_api_key = credential.ALPHA_VANTAGE_API_KEY

### Data Preparation

In [4]:
# Download stock data from yfinance and alpha_vantage
yfinance_data = pd.concat([download_yfinance_data(ticker, start_date, end_date) for ticker in tickers])
alpha_vantage_data = pd.concat([download_alpha_vantage_data(ticker, alpha_vantage_api_key) for ticker in tickers])

[*********************100%***********************]  1 of 1 completed


In [5]:
alpha_vantage_data

,str_date,Open,High,Low,Close,Adj Close,Volume,Stock,Source
0,2024-11-15,207.46,208.49,204.07,204.99,204.99,3986460,IBM,alpha vantage
1,2024-11-14,210.0,210.4999,206.35,208.99,208.99,6372853,IBM,alpha vantage
2,2024-11-13,209.5,211.41,209.0701,210.92,210.92,3247830,IBM,alpha vantage
3,2024-11-12,211.9,213.03,209.06,210.86,210.86,2818216,IBM,alpha vantage
4,2024-11-11,214.4,215.41,213.48,213.57,211.891827977227,3012987,IBM,alpha vantage
...,...,...,...,...,...,...,...,...,...
6297,1999-11-05,92.75,92.94,90.19,90.25,45.7112254831793,13737600,IBM,alpha vantage
6298,1999-11-04,94.44,94.44,90.0,91.56,46.3747346840985,16697600,IBM,alpha vantage
6299,1999-11-03,95.87,95.94,93.5,94.37,47.7979872448491,10369100,IBM,alpha vantage
6300,1999-11-02,96.75,96.81,93.69,94.81,48.0208452970662,11105400,IBM,alpha vantage


In [6]:
yfinance_data

Price,Date,Adj Close,Close,High,Low,Open,Volume,Stock,Source
0,2023-01-03 00:00:00+00:00,130.147675,141.550003,141.899994,140.479996,141.100006,3338600,IBM,yfinance
1,2023-01-04 00:00:00+00:00,131.113113,142.600006,143.619995,141.369995,142.070007,3869200,IBM,yfinance
2,2023-01-05 00:00:00+00:00,129.743134,141.110001,142.500000,140.009995,142.440002,2866600,IBM,yfinance
3,2023-01-06 00:00:00+00:00,132.124496,143.699997,144.250000,141.580002,142.380005,3574000,IBM,yfinance
4,2023-01-09 00:00:00+00:00,131.986588,143.550003,145.470001,143.399994,144.080002,3987700,IBM,yfinance
...,...,...,...,...,...,...,...,...,...
245,2023-12-22 00:00:00+00:00,156.483139,162.139999,162.410004,161.000000,161.100006,2439800,IBM,yfinance
246,2023-12-26 00:00:00+00:00,157.515808,163.210007,163.309998,162.050003,162.229996,1772400,IBM,yfinance
247,2023-12-27 00:00:00+00:00,157.757095,163.460007,163.639999,162.679993,163.139999,3234600,IBM,yfinance
248,2023-12-28 00:00:00+00:00,158.036972,163.750000,163.960007,163.399994,163.960007,2071300,IBM,yfinance


### PySpark Manipulation

In [7]:
# Load Data to PySpark
spark = SparkSession.builder.config("spark.executor.memory", "4g").config("spark.driver.memory", "2g").config("spark.network.timeout", "600s").appName("Stock Data Preparation").getOrCreate()

yfinance_df = spark.createDataFrame(yfinance_data)
alpha_vantage_df = spark.createDataFrame(alpha_vantage_data)

In [8]:
yfinance_df.printSchema()
alpha_vantage_df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- Volume: long (nullable = true)
 |-- Stock: string (nullable = true)
 |-- Source: string (nullable = true)

root
 |-- str_date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Adj Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Stock: string (nullable = true)
 |-- Source: string (nullable = true)



In [9]:
# Align the data type of the two PySpark Dataframes
yfinance_df = yfinance_df.withColumn("Date", to_date(col("Date"), "yyyy-MM-dd"))

alpha_vantage_df = alpha_vantage_df.withColumn("Date", to_date(col("str_date"), "yyyy-MM-dd"))
alpha_vantage_df = alpha_vantage_df.drop('str_date')

double_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close']
for double_col in double_cols:
    alpha_vantage_df = alpha_vantage_df.withColumn(double_col, col(double_col).cast("double"))

alpha_vantage_df = alpha_vantage_df.withColumn("Volume", col("Volume").cast("long"))

In [16]:
# Union the dataframe of 2 source to one
full_df = yfinance_df.select(sorted(yfinance_df.columns)).union(alpha_vantage_df.select(sorted(alpha_vantage_df.columns)))

In [17]:
# Remove na and duplicate values, filter the period for analysis
full_df = full_df.na.drop().dropDuplicates()
full_df = full_df.filter((full_df.Date >= start_date) & (full_df.Date <= end_date))

In [18]:
# Add Year and Month columns for aggregation
full_df = full_df.withColumn("Year", year(col("Date")))
full_df = full_df.withColumn("Month", month(col("Date")))
full_df.show(5)

+------------------+------------------+----------+------------------+------------------+------------------+--------+-----+-------+----+-----+
|         Adj Close|             Close|      Date|              High|               Low|              Open|  Source|Stock| Volume|Year|Month|
+------------------+------------------+----------+------------------+------------------+------------------+--------+-----+-------+----+-----+
|127.83721160888672|137.35000610351562|2023-02-13|137.38999938964844|135.85000610351562|             136.0|yfinance|  IBM|4403000|2023|    2|
| 126.9530029296875|136.39999389648438|2023-02-15| 136.4499969482422|135.07000732421875| 135.1999969482422|yfinance|  IBM|2506700|2023|    2|
| 129.4213104248047|140.75999450683594|2023-01-25|141.02999877929688|139.36000061035156|140.47000122070312|yfinance|  IBM|7347500|2023|    1|
|129.09951782226562|140.41000366210938|2023-01-18|144.67999267578125|140.22999572753906|144.39999389648438|yfinance|  IBM|6445600|2023|    1|
|124.8

In [19]:
# Aggregate result
agg_df = full_df.groupBy("Source", "Stock", "Year", "Month").agg(
    spark_avg("Close").alias("Avg_Close"),
    spark_sum("Volume").alias("Total_Volume")
).orderBy(['Stock', 'Year', 'Month', 'Source'])

agg_df.show()

+-------------+-----+----+-----+------------------+------------+
|       Source|Stock|Year|Month|         Avg_Close|Total_Volume|
+-------------+-----+----+-----+------------------+------------+
|alpha vantage|  IBM|2023|    1|141.22050000000002|   105576019|
|     yfinance|  IBM|2023|    1|141.22050018310546|   105575000|
|alpha vantage|  IBM|2023|    2|134.17789473684212|    76080679|
|     yfinance|  IBM|2023|    2|134.17789499383224|    76080200|
|alpha vantage|  IBM|2023|    3|126.98478260869568|   138093084|
|     yfinance|  IBM|2023|    3|126.98478300675102|   138239000|
|alpha vantage|  IBM|2023|    4| 128.2557894736842|    83664114|
|     yfinance|  IBM|2023|    4|128.25578950580797|    83679400|
|alpha vantage|  IBM|2023|    5|125.10318181818184|    95710890|
|     yfinance|  IBM|2023|    5|125.10318097201261|    96207800|
|alpha vantage|  IBM|2023|    6|133.92000000000002|   100722016|
|     yfinance|  IBM|2023|    6|133.91999962216332|   100711500|
|alpha vantage|  IBM|2023

In [21]:
# Use Spark SQL, validate data from 2 difference sources
full_df.createOrReplaceTempView("stock_price_validation")

query = """
SELECT CASE
        WHEN yf.Date is NULL THEN av.Date
        ELSE yf.Date
    END AS Date
    , CASE
        WHEN yf.Stock IS NULL THEN av.Stock
        ELSE yf.Stock
    END AS Stock
    , yf.Close as yf_Price, av.Close as alphavantage_Price
    , CASE
        WHEN yf.Close IS NULL and av.Close IS NOT NULL THEN "yfinance missing data"
        WHEN av.Close IS NULL and yf.Close IS NOT NULL THEN "alpha vantage missing data"
        WHEN ROUND(yf.Close, 2) != ROUND(av.Close, 2) THEN "price conflict"
        ELSE "validated data"
    END AS Check_Status
FROM (SELECT Date, Stock, Close FROM stock_price_validation WHERE Source = 'yfinance') yf
FULL OUTER JOIN (SELECT Date, Stock, Close FROM stock_price_validation WHERE Source = 'alpha vantage') av
ON yf.Date = av.Date AND yf.Stock = av.stock
ORDER BY (CASE
            WHEN yf.Date is NULL THEN av.Date
            ELSE yf.Date
        END)
"""
query_result = spark.sql(query)
query_result.show()


+----------+-----+------------------+------------------+--------------+
|      Date|Stock|          yf_Price|alphavantage_Price|  Check_Status|
+----------+-----+------------------+------------------+--------------+
|2023-01-03|  IBM| 141.5500030517578|            141.55|validated data|
|2023-01-04|  IBM|142.60000610351562|             142.6|validated data|
|2023-01-05|  IBM|141.11000061035156|            141.11|validated data|
|2023-01-06|  IBM| 143.6999969482422|             143.7|validated data|
|2023-01-09|  IBM| 143.5500030517578|            143.55|validated data|
|2023-01-10|  IBM| 144.8000030517578|             144.8|validated data|
|2023-01-11|  IBM|145.25999450683594|            145.26|validated data|
|2023-01-12|  IBM| 145.5500030517578|            145.55|validated data|
|2023-01-13|  IBM|145.88999938964844|            145.89|validated data|
|2023-01-17|  IBM|145.19000244140625|            145.19|validated data|
|2023-01-18|  IBM|140.41000366210938|            140.41|validate

In [23]:
# Write the output to Parquet
query_result.write.parquet("data/stock_price_validation.parquet")
agg_df.write.parquet("data/stock_agg_by_month.parquet")

In [24]:
spark.stop()